# MSA 2025 Phase 2 - Part 1

In [1]:
import sklearn
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Find all variables and understand them

In [15]:
#
sales = pd.read_csv("data/raw/sales.csv")
print(sales.head(10))
sales.describe().round(1)


   Store  Dept        Date  Weekly_Sales  IsHoliday
0      1     1  2010-02-05      24924.50      False
1      1     1  2010-02-12      46039.49       True
2      1     1  2010-02-19      41595.55      False
3      1     1  2010-02-26      19403.54      False
4      1     1  2010-03-05      21827.90      False
5      1     1  2010-03-12      21043.39      False
6      1     1  2010-03-19      22136.64      False
7      1     1  2010-03-26      26229.21      False
8      1     1  2010-04-02      57258.43      False
9      1     1  2010-04-09      42960.91      False


,Store,Dept,Weekly_Sales
count,421570.0,421570.0,421570.0
mean,22.2,44.3,15981.3
std,12.8,30.5,22711.2
min,1.0,1.0,-4988.9
25%,11.0,18.0,2079.6
50%,22.0,37.0,7612.0
75%,33.0,74.0,20205.9
max,45.0,99.0,693099.4


In [17]:
features = pd.read_csv("data/raw/features.csv")
print(features.head(10))
features.describe().round(1)

   Store        Date  Temperature  Fuel_Price  MarkDown1  MarkDown2  \
0      1  2010-02-05        42.31       2.572        NaN        NaN   
1      1  2010-02-12        38.51       2.548        NaN        NaN   
2      1  2010-02-19        39.93       2.514        NaN        NaN   
3      1  2010-02-26        46.63       2.561        NaN        NaN   
4      1  2010-03-05        46.50       2.625        NaN        NaN   
5      1  2010-03-12        57.79       2.667        NaN        NaN   
6      1  2010-03-19        54.58       2.720        NaN        NaN   
7      1  2010-03-26        51.45       2.732        NaN        NaN   
8      1  2010-04-02        62.27       2.719        NaN        NaN   
9      1  2010-04-09        65.86       2.770        NaN        NaN   

   MarkDown3  MarkDown4  MarkDown5         CPI  Unemployment  IsHoliday  
0        NaN        NaN        NaN  211.096358         8.106      False  
1        NaN        NaN        NaN  211.242170         8.106       Tru

,Store,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
count,8190.0,8190.0,8190.0,4032.0,2921.0,3613.0,3464.0,4050.0,7605.0,7605.0
mean,23.0,59.4,3.4,7032.4,3384.2,1760.1,3292.9,4132.2,172.5,7.8
std,13.0,18.7,0.4,9262.7,8793.6,11276.5,6792.3,13086.7,39.7,1.9
min,1.0,-7.3,2.5,-2781.4,-265.8,-179.3,0.2,-185.2,126.1,3.7
25%,12.0,45.9,3.0,1577.5,68.9,6.6,304.7,1440.8,132.4,6.6
50%,23.0,60.7,3.5,4743.6,364.6,36.3,1176.4,2727.1,182.8,7.8
75%,34.0,73.9,3.7,8923.3,2153.4,163.2,3310.0,4832.6,213.9,8.6
max,45.0,102.0,4.5,103185.0,104519.5,149483.3,67474.8,771448.1,229.0,14.3


In [ ]:
stores = pd.read_csv("data/raw/stores.csv")
print(stores.head(10))
print(stores["Type"].value_counts())
stores.describe().round(1)

   Store Type    Size
0      1    A  151315
1      2    A  202307
2      3    B   37392
3      4    A  205863
4      5    B   34875
5      6    A  202505
6      7    B   70713
7      8    A  155078
8      9    B  125833
9     10    B  126512
A    22
B    17
C     6
Name: Type, dtype: int64


,Store,Size
count,45.0,45.0
mean,23.0,130287.6
std,13.1,63825.3
min,1.0,34875.0
25%,12.0,70713.0
50%,23.0,126512.0
75%,34.0,202307.0
max,45.0,219622.0


## 2. Clean data

In [3]:
#

## 3. Visualise data

In [4]:
#

## 4. Identify correlated variables

In [5]:
#

## 5. Summary